<a href="https://colab.research.google.com/github/Thanos002/Anwendungen_KI/blob/main/Bonus/Bonus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install necessary libraries:

In [3]:
pip install transformers datasets evaluate accelerate transformers[torch] pyspellchecker emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.7 MB/s eta 0:00:00


Import Libraries


In [4]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import torch
import html
import numpy as np
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from sklearn.utils.class_weight import compute_class_weight
from spellchecker import SpellChecker
import emoji

import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import words as nltk_words
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')



Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

Check GPU availability:

In [5]:
# Set the device to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


Abbreviaation list

In [6]:
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk",
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart",
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", #"que pasa",
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet",
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
    "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously",
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

Preprocessing

In [7]:
def preprocess_text(text):
    """
    Function to preprocess text data.
    """

    # Remove 'RT' and mentions like '@username'
    text = re.sub(r'\bRT\s?@\w+|\bRT\b|@\w+', '', text)

    # Convert text to lowercase
    text = text.lower()

    # Remove consecutive identical HTML entities for emojis
    text = re.sub(r'(&#\d+;)(\1)+', r'\1', text)

    # Unescape HTML entities (including emojis)
    text = html.unescape(text)

    # Convert Unicode emojis to their text representation
    text = emoji.demojize(text, delimiters=(" ", " "))

    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove HTML tags (if any remain after unescaping)
    text = re.sub(r'<.*?>', '', text)

    #Replace lowerscores with blanks
    text = text.replace('_', ' ')

    # Modify hashtag removal: remove only hashtags without letters
    text = re.sub(r'#\b\w+\b', '', text)

    #Remove hashtag character only
    text = re.sub(r'#(?=\w)', '', text)

    """
    # Replace common abbreviations
    for abbr, full_form in abbreviations.items():
        text = text.replace(abbr, full_form)
    """
    """
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))
    """
    # Remove quotes
    text = text.replace('"', '')

    #Remove newline character
    text = text.replace('\n', '')

    # Remove doule space
    text = text.replace('  ', ' ')

    # Remove all special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    """
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    """
    """
    # Spell checking with pre-filtering
    words_list = nltk_words.words()
    spell = SpellChecker()
    processed_words = []
    for word in text.split():
        if word not in words_list:
            corrected_word = spell.correction(word)
            print(word, " to ", corrected_word)
            # Append the corrected word if it's not None, otherwise append the original word
            processed_words.append(corrected_word if corrected_word is not None else word)
        else:
            processed_words.append(word)

    text = ' '.join(processed_words)
    """
    """
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    text = ' '.join([word for word in words if word not in stop_words])

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    """
    return text


Load Dataset

In [8]:
# Load the dataset
file_path = '/content/drive/My Drive/train.csv'  # Update with the correct path
data = pd.read_csv(file_path)

data['text'] = data['text'].apply(preprocess_text)

# Display the first few rows of the dataset
print(data.head(n=20))


    Unnamed: 0                                               text  label
0            0    la  palm trees and great weather any were el...      1
1            1   the face you make when you see a hoe trying t...      1
2            2        bitch get off my twitter hoe oncoming fist       1
3            3  i can taste loud n pussy on my tongue squintin...      1
4            4  diabetes galore face with tears of joy  colora...      2
5            5                you sick bitch didnt even invite me      1
6            6   never hurt your girlfriend to make a hoe happ...      1
7            7  this is why nothing gets done morons like cobu...      2
8            8   miserable bitches talk the most shit face wit...      1
9            9  its gorg out  this bitch thinks its june or so...      1
10          10  know why the ultimate warrior died god loves f...      0
11          11                     she looks like a real dyke now      1
12          12                                   fi

Define a Dataset class for handling the Twitter comments. This involves tokenizing the text and preparing it in a format suitable for BERT.

In [9]:
class TwitterCommentsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [10]:
df_train, df_val = train_test_split(data, test_size=0.15)


In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 256  # You can adjust this depending on the average length of the tweets

train_dataset = TwitterCommentsDataset(
    texts=df_train.text.to_numpy(),
    labels=df_train.label.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
)

val_dataset = TwitterCommentsDataset(
    texts=df_val.text.to_numpy(),
    labels=df_val.label.to_numpy(),
    tokenizer=tokenizer,
    max_len=max_len
)


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Load BERT Model with class weights

In [12]:
# Calculate class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(df_train['label']),
    y=df_train['label'])

# Convert class weights to tensor
weights = torch.tensor(class_weights, dtype=torch.float)
weights = weights.to(device)

# Modify your model to use these weights in the loss function
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data.label.unique()))
model.classifier = torch.nn.Linear(model.classifier.in_features, len(np.unique(df_train['label'])))
model.num_labels = len(np.unique(df_train['label']))
model.to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [13]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=64,
    learning_rate=5e-5,              # initial learning rate for Adam optimizer
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=200,
    evaluation_strategy='steps',     # evaluate during fine-tuning so that we can see progress
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Trainer setup

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)


Training and Evaluation

In [15]:
trainer.train()

results = trainer.evaluate()
print(results)

# Assuming 'trainer' is your Trainer object from the transformers library
predictions = trainer.predict(val_dataset)

# Get the predicted labels
predicted_labels = predictions.predictions.argmax(-1)

# Assuming 'true_labels' is a list or array containing your true labels
print(classification_report(val_dataset.labels, predicted_labels))



Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
200,0.585300,0.296593,0.898117,0.880175,0.883173,0.898117
400,0.306100,0.268777,0.909213,0.880668,0.853861,0.909213
600,0.291000,0.258281,0.908877,0.888103,0.890107,0.908877
800,0.267400,0.257490,0.907196,0.887973,0.886835,0.907196
1000,0.249600,0.268530,0.908877,0.885689,0.898759,0.908877
1200,0.248400,0.244032,0.912912,0.911523,0.910576,0.912912
1400,0.236400,0.242109,0.910222,0.911231,0.912338,0.910222
1600,0.165000,0.294287,0.911231,0.909337,0.907993,0.911231
1800,0.169200,0.249343,0.918628,0.915510,0.913511,0.918628
2000,0.166100,0.261762,0.917956,0.909506,0.907562,0.917956


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.26509514451026917, 'eval_accuracy': 0.9176193678547411, 'eval_f1': 0.9092449542225433, 'eval_precision': 0.9071481099735127, 'eval_recall': 0.9176193678547411, 'eval_runtime': 47.6772, 'eval_samples_per_second': 62.378, 'eval_steps_per_second': 0.986, 'epoch': 3.0}
              precision    recall  f1-score   support

           0       0.58      0.29      0.39       181
           1       0.94      0.97      0.95      2284
           2       0.88      0.91      0.90       509

    accuracy                           0.92      2974
   macro avg       0.80      0.72      0.75      2974
weighted avg       0.91      0.92      0.91      2974



NameError: ignored

Overview of labels that have been labelled incorrectly:

In [16]:

# Function to make predictions on validation set
def get_predictions(model, data_loader):
    model = model.eval()
    predictions = []
    real_values = []

    with torch.no_grad():
        for d in data_loader:
            inputs = {
                'input_ids':      d["input_ids"].to(device),
                'attention_mask': d["attention_mask"].to(device),
                'labels':         d["labels"].to(device)
            }

            outputs = model(**inputs)
            logits = outputs.logits
            _, preds = torch.max(logits, dim=1)

            predictions.extend(preds)
            real_values.extend(inputs['labels'])

    predictions = torch.stack(predictions).cpu()
    real_values = torch.stack(real_values).cpu()
    return predictions, real_values

# Create a DataLoader for the validation set
val_data_loader = DataLoader(
    val_dataset,
    batch_size=training_args.per_device_eval_batch_size
)

# Get predictions and real labels
predictions, real_values = get_predictions(model, val_data_loader)

# Compare predictions with true labels
misclassified = []
for idx, (pred, true) in enumerate(zip(predictions, real_values)):
    if pred != true:
        misclassified.append((idx, df_val.iloc[idx].text, true.item(), pred.item()))

# Output the misclassified data
for idx, text, true_label, pred_label in misclassified:
    print(f"Index: {idx}, Comment: {text}, True Label: {true_label}, Predicted Label: {pred_label}")


Index: 3, Comment:  he looks like reg memphistn trash we got them everywhere, True Label: 0, Predicted Label: 2
Index: 10, Comment: why wont we run the ball our offensive coordinator is retarded, True Label: 2, Predicted Label: 1
Index: 31, Comment: chocolate cake is so trash, True Label: 1, Predicted Label: 2
Index: 44, Comment:  honestly thats where im the most retarded and is why stuck as artist hoping for a fair dealermanager situation, True Label: 2, Predicted Label: 1
Index: 50, Comment: i hate you niccas lol , True Label: 0, Predicted Label: 1
Index: 68, Comment:    metal monkey balls, True Label: 1, Predicted Label: 2
Index: 79, Comment: uhuh what do we have your empty heart my empty bottle a yellow cab that you would like to catch cuz yo latest ex made u mad, True Label: 1, Predicted Label: 2
Index: 97, Comment:  stfu dyke looking hoe, True Label: 0, Predicted Label: 1
Index: 98, Comment:    jessica im not afraid to slit your throat tomorrow  , True Label: 0, Predicted Label: 

In [32]:

import emoji
import html
text = "Lightskingawdd: Wish I had a bae &#128533 Python is 👍"
text = html.unescape(text)
print(emoji.demojize(text))

Lightskingawdd: Wish I had a bae :confused_face: Python is :thumbs_up:
